<a href="https://colab.research.google.com/github/navidadkhah/Fine-Tuning-LLMs/blob/main/Evaluation/CodeBELU_metric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialize Libraries

In [1]:
!pip install codebleu==0.1.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.6/566.6 kB 30.8 MB/s eta 0:00:00


In [2]:
from codebleu import calc_codebleu
from codebleu import calc_codebleu
from nltk.translate.bleu_score import sentence_bleu

import numpy as np
import pandas as pd

# Reading the Datasets



In [3]:
path = "/content/drive/MyDrive/Bachelor\'s project/bug_evaluation_dataset.csv"
df = pd.read_csv(path)
print(df.shape)
df.head()

(25793, 5)


,original_code,modified_code,changed_line,number_of_line,mutation_type
0,n = int(input())\nw = [2] * n\nans0 = n\nans =...,n = int(input())\nw = [2] * n\nans0 = n\nans =...,"for j in range(y-1, r):",14,statement
1,n = int(input())\nw = [2] * n\nans0 = n\nans =...,n = int(input())\nw = [2] * n\nans0 = n\nans =...,if k<1:,8,decision
2,n = int(input())\nw = [2] * n\nans0 = n\nans =...,n = int(input())\nw = [2] * n\nans0 = n\nans =...,if k<1:,8,decision
3,n = int(input())\nw = [2] * n\nans0 = n\nans =...,n = int(input())\nw = [2] * n\nans0 = n\nans =...,"for j in range(l&1, r):",14,operation
4,n = int(input())\nw = [2] * n\nans0 = n\nans =...,n = int(input())\nw = [2] * n\nans0 = n\nans =...,"l, r, k = list(z(int, input().split()))",7,statement


In [67]:
path = "/content/drive/MyDrive/Bachelor\'s project/llm_response_checkpoint_comments_1000.csv"
commented_prompt_df = pd.read_csv(path)
print(commented_prompt_df.shape)
commented_prompt_df.head()

(1000, 1)


,simple_prompt
0,Here is the code with comments:\n\n```\nn = in...
1,Here is the code with comments:\n\n```\nn = in...
2,Here is the code with comments:\n\n```\nn = in...
3,Here is the code with comments:\n\n```\nn = in...
4,Here is the code with comments:\n\n```\nn = in...


In [75]:
path = "/content/drive/MyDrive/Bachelor\'s project/llm_response_checkpoint_3000.csv"
simple_prompt_df = pd.read_csv(path)
print(simple_prompt_df.shape)
simple_prompt_df.head()

(3000, 1)


,simple_prompt
0,Here is the corrected code:\n\n```\nn = int(in...
1,```\nn = int(input())\nw = [2] * n\nans0 = n\n...
2,```\nn = int(input())\nw = [2] * n\nans = [0] ...
3,```\nn = int(input())\nw = [2] * n\nans0 = n\n...
4,```\nn = int(input())\nw = [2] * n\nans0 = n\n...


# Preprocessing the dataset

In [68]:
def generate_code_from_list(lines):
    # Join the list of strings into a single code block
    code_block = "\n".join(lines)

    # Remove any lines before and after the code block
    code_block = code_block.split('```')[1].split('```')[0].strip()

    # Replace the placeholder 'x' with a proper definition (e.g., 'a')
    code_block = code_block.replace("x", "a")  # Fixing the undefined variable bug

    # Remove comments (inline and block)
    code_block = "\n".join([
        line.split("#")[0].strip()  # Keep only the part before the inline comment (if any)
        for line in code_block.splitlines()
        if line.strip() and not line.strip().startswith("#")  # Exclude full-line comments
    ])

    return code_block



# Iterate over each row in the DataFrame
for row in range(len(commented_prompt_df)):
    # Split the 'simple_prompt' into lines
    input_lines = commented_prompt_df['simple_prompt'][row].split("\n")

    # # Generate the corrected code block without comments
    generated_code = generate_code_from_list(input_lines)

    # # Save the generated code into a new column
    commented_prompt_df.at[row, 'generated_code'] = generated_code

# Now 'simple_prompt_df' will have a 'generated_code' column with the results
commented_prompt_df.head()

,simple_prompt,generated_code
0,Here is the code with comments:\n\n```\nn = in...,n = int(input())\nw = [2] * n\nans0 = n\nans =...
1,Here is the code with comments:\n\n```\nn = in...,n = int(input())\nw = [2] * n\nans0 = n\nans =...
2,Here is the code with comments:\n\n```\nn = in...,n = int(input())\nw = [2] * n\nans0 = n\nans =...
3,Here is the code with comments:\n\n```\nn = in...,n = int(input())\nw = [2] * n\nans0 = n\nans =...
4,Here is the code with comments:\n\n```\nn = in...,n = int(input())\nw = [2] * n\nans0 = n\nans =...


# CodeBELU metric

In [70]:
def codeBELU_metric(pred, ref):
  # Tokenize the code for BLEU calculation
  pred_tokens = pred.split()
  ref_tokens = [ref.split()]  # BLEU expects a list of references

  # Calculate the BLEU score
  bleu_score = sentence_bleu(ref_tokens, pred_tokens)
  return bleu_score

In [77]:
# Calculate BLEU scores for all pairs
bleu_scores = [codeBELU_metric(pred, ref) for pred, ref in zip(commented_prompt_df['generated_code'], df['original_code'])]
# # Compute the mean of BLEU scores
mean_bleu_score = sum(bleu_scores) / len(bleu_scores)
print(f"Mean BLEU Score: {mean_bleu_score}")

Mean BLEU Score: 0.6318759252961571


In [78]:
# Calculate BLEU scores for all pairs
bleu_scores = [codeBELU_metric(pred, ref) for pred, ref in zip(simple_prompt_df['simple_prompt'], df['original_code'])]
# # Compute the mean of BLEU scores
mean_bleu_score = sum(bleu_scores) / len(bleu_scores)
print(f"Mean BLEU Score: {mean_bleu_score}")

Mean BLEU Score: 0.6589338331510101
